In [0]:
service_credential = dbutils.secrets.get(scope="azure-key-vault-secret1",key="datalake-secret")

spark.conf.set("fs.azure.account.auth.type.rkudatalakegen2.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.rkudatalakegen2.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.rkudatalakegen2.dfs.core.windows.net", "e1c699e2-256e-4508-a129-340636ed905f")
spark.conf.set("fs.azure.account.oauth2.client.secret.rkudatalakegen2.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.rkudatalakegen2.dfs.core.windows.net", "https://login.microsoftonline.com/6dfaeccd-2d4f-43fc-8db8-8298cf6e47f1/oauth2/token")

Transforming data in all tables

In [0]:
table_name=[]

for i in dbutils.fs.ls("abfss://bronze@rkudatalakegen2.dfs.core.windows.net/SalesLT"):
    table_name.append(i.name.split('/')[0])

print(table_name)

['Address', 'Customer', 'CustomerAddress', 'Product', 'ProductCategory', 'ProductDescription', 'ProductModel', 'ProductModelProductDescription', 'SalesOrderDetail', 'SalesOrderHeader']


In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

for n in table_name:
    path=f"abfss://bronze@rkudatalakegen2.dfs.core.windows.net/SalesLT/{n}/{n}.parquet"
    df = spark.read.format('parquet').load(path)
    columns= df.columns
    
    for col in columns:
        if "Date" in col or "date" in col:
            df = df.withColumn(col,date_format(from_utc_timestamp(df[col].cast(TimestampType()),"UTC"),"yyyy-mm-dd"))

    output_path = f"abfss://silver@rkudatalakegen2.dfs.core.windows.net/SalesLT/{n}/"
    #df.write.format('delta').mode("overwrite").save(output_path)
    df.write.format('delta').option("overwriteSchema", "true").option("overwriteDataSchema", "true").option("checkConstraints", "false").option("ignoreDeletes", "true").option("ignoreChanges", "true").option("mergeSchema", "true").mode("overwrite").save(output_path)
